In [8]:
#爬蟲功能
#從tripadvosor網站中爬出評論
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [9]:
html = urlopen("https://www.tripadvisor.com/Restaurant_Review-g60763-d3163891-Reviews-Serafina_77th_Upper_West-New_York_City_New_York.html")
bsObj = BeautifulSoup(html,'html.parser')

In [10]:
#查看總共有幾頁
# totalreview = bsObj.find("div",{"class":"pagination-details"}).findAll("b")[2].get_text()
# #超過1000會有1,000
# totalreview = int(totalreview.split()[0].replace(',', ''))
# page = totalreview/10+1
# pages= int(page)

In [11]:
# -*- coding: utf-8 -*-
#檢查是否符合分句子標準
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(r'!+',"!",text)
    text = re.sub(r'\?+',"?",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace("!?","?")
    text = text.replace("?!","!")
    text = text.replace("..",".")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [14]:
def write(string):
    with open('./original_review_sets/review'+str(count)+'.txt','w', encoding = 'UTF8') as f :  #存入哪個資料夾
        f.write(string)
        f.close()

In [ ]:
index=''
count=1 #1
for x in range (20):#pages 20頁=200則評論
    if(x==0):
        index='-'
    else:
        index="-or"+str(x*10)+'-'
    url2 = "https://www.tripadvisor.com/Restaurant_Review-g60763-d3163891-Reviews"+index+"Serafina_77th_Upper_West-New_York_City_New_York.html"
    html = urlopen(url2)
    bsObj = BeautifulSoup(html,'html.parser')
    reviewlist = bsObj.findAll("div",{"class":"review-container"})
    for review in reviewlist:
        url = review.find("a",{"class":"title"})
        head = "https://www.tripadvisor.com/"
        tail = url.get("href")
        html2 = urlopen(head+tail)
        bsObj = BeautifulSoup(html2,'html.parser')

        string = bsObj.find("p",{"class":"partial_entry"}).get_text()
        
        string_list = split_into_sentences(string) #split the text into sentences
        for x in range(len(string_list)):
            string = string_list[x]
            write(string)
            count+=1

